In [1]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

# Toggle cell visibility

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

# Hide the code completely

# from IPython.display import HTML
# tag = HTML('''<style>
# div.input {
#     display:none;
# }
# </style>''')
# display(tag)


## Modal analysis of the lunar lander

The dynamic matrix $A$ that represents the Lunar Lander dynamics is (see example [Lunar lander lateral position dynamics](SS-11-Lunar_lander_lateral_position_dynamics) for more details):

$$
A=\begin{bmatrix}0&1&0&0 \\ 0&0&F/m&0 \\ 0&0&0&1 \\ 0&0&0&0\end{bmatrix},
$$

where $F$ is the thrust force and $m$ the mass of the lander. The state of the system is $x=[z,\dot{z},\theta,\dot{\theta}]^T$, where $z$ is the lateral position, $\dot{z}$ the time variation of the lateral position, $\theta$ the orientation angle of the lander with respect the vertical and $\dot{\theta}$ its variation in time.

The dynamic matrix in this form shows four eigenvalues, all equal to 0. Eigenvalues 0 are often called integrators (recall the Laplace Transform of a the integral of a signal: what is the root of the denominator of its expression?), thus this system is said to have 4 integrators. With $F\neq0$ ($m\neq0$) the system presents a structure that is similar to a $4\times4$ Jordan block, so the eigenvalue 0, in this case, has a geometrical multiplicity equal to 1. With $F=0$ the eigenvalue remains the same with the same algebraic multiplicity but with a geometrical multiplicity equal to 2.

Presented below is an example with $F\neq0$.

### How to use this notebook?

- Try to set $F=0$ and try to explain what physically implies this case for the lander, specially for the $z$ and $\theta$ dynamics and their relationship.

In [2]:
#Preparatory Cell 
import control
import numpy
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt

%matplotlib inline

#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [3]:
#define the sliders for m, k and c
m = widgets.FloatSlider(
    value=1000,
    min=400,
    max=2000,
    step=1,
    description='$m$ [kg]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
F = widgets.FloatSlider(
    value=1500,
    min=0,
    max=5000,
    step=10,
    description='$F$ [N]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [4]:
#function that make all the computations
def main_callback(m, F):
    eig1 = 0
    eig2 = 0
    eig3 = 0
    eig4 = 0
    
    if numpy.real([eig1,eig2,eig3,eig4])[0] == 0 and numpy.real([eig1,eig2,eig3,eig4])[1] == 0:
        T = numpy.linspace(0,20,1000)
    else:
        if min(numpy.abs(numpy.real([eig1,eig2,eig3,eig4]))) != 0:
            T = numpy.linspace(0,7*1/min(numpy.abs(numpy.real([eig1,eig2,eig3,eig4]))),1000)
        else:
            T = numpy.linspace(0,7*1/max(numpy.abs(numpy.real([eig1,eig2,eig3,eig4]))),1000)
    if F==0:
        mode1 = numpy.exp(eig1*T)
        mode2 = T*mode1
        mode3 = mode1
        mode4 = mode2
    else:
        mode1 = numpy.exp(eig1*T)
        mode2 = T*mode1
        mode3 = T*mode2
        mode4 = T*mode3
    
    fig = plt.figure(figsize=[16, 10])
    fig.set_label('Modes')
    g1 = fig.add_subplot(221)
    g2 = fig.add_subplot(222)
    g3 = fig.add_subplot(223)
    g4 = fig.add_subplot(224)
    
    g1.plot(T,mode1)
    g1.grid()
    g1.set_xlabel('Time [s]')
    g1.set_ylabel('First mode')
    
    g2.plot(T,mode2)
    g2.grid()
    g2.set_xlabel('Time [s]')
    g2.set_ylabel('Second mode')
    
    g3.plot(T,mode3)
    g3.grid()
    g3.set_xlabel('Time [s]')
    g3.set_ylabel('Third mode')
    
    g4.plot(T,mode4)
    g4.grid()
    g4.set_xlabel('Time [s]')
    g4.set_ylabel('Fourth mode')
    
    modesString = r'The eigenvalue is equal to 0 with algebraic multiplicity equal to 4. '
    if F==0:
        modesString = modesString + r'The corresponding modes are $k$ and $t$.'
    else:
        modesString = modesString + r'The corresponding modes are $k$, $t$, $\frac{t^2}{2}$ and $\frac{t^3}{6}$.'
    display(Markdown(modesString))

        
out = widgets.interactive_output(main_callback,{'m':m,'F':F})
sliders = widgets.HBox([m,F])
display(out,sliders)

Output()